In [8]:
import os
import pandapower as pp
import pandas as pd
import numpy as np
import polars as pl
from polars import col as c
import pyomo.environ as pyo
from data_display.grid_plotting import plot_grid_from_pandapower
from data_connector import pandapower_to_dig_a_plan_schema
from dig_a_plan_pipeline import DigAPlan
from pyomo_utility import extract_optimization_results
from pyomo.opt import SolverFactory
os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = os.environ["HOME"] + "/gurobi_license/gurobi.lic"

In [10]:


# 1. Create a ConcreteModel
model = pyo.ConcreteModel()

# 2. Define Variables
model.x = pyo.Var(bounds=(0, None), initialize=0)
model.y = pyo.Var(bounds=(0, None), initialize=0)

# 3. Define the Objective Function (Maximize 3x + 2y)
model.obj = pyo.Objective(expr=3 * model.x - 2 * model.y, sense=pyo.minimize)

# 4. Define Constraints
model.constraint1 = pyo.Constraint(expr=model.x + model.y <= 10)
model.constraint2 = pyo.Constraint(expr=2 * model.x + model.y >= 15)

# IMPORTANT: Declare the 'dual' Suffix to tell Pyomo to import duals
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# 5. Choose Gurobi as the Solver
solver_name = 'gurobi'
opt = SolverFactory(solver_name)

# 6. Solve the model
results = opt.solve(model, tee=False)  # tee=True shows solver output

# 7. Load results back into the model
# model.load_results(results)

# 8. Check solver status and access dual values
if (results.solver.status == pyo.SolverStatus.ok and
    results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print("\n--- Optimal Solution Found ---")
    print(f"Objective Value: {pyo.value(model.obj)}")
    print(f"x = {pyo.value(model.x)}")
    print(f"y = {pyo.value(model.y)}")

    print("\n--- Dual Values (Shadow Prices) ---")
    for c in model.component_objects(pyo.Constraint, active=True):
        print(f"Constraint: {c.name}")
        for index in c:
            dual_value = model.dual[c[index]]
            print(f"  Index: {index}, Dual Value: {dual_value}")

else:
    print("\n--- Solver did not find an optimal solution ---")
    print(f"Solver Status: {results.solver.status}")
    print(f"Termination Condition: {results.solver.termination_condition}")


--- Optimal Solution Found ---
Objective Value: 5.0
x = 5.0
y = 5.0

--- Dual Values (Shadow Prices) ---
Constraint: constraint1
  Index: None, Dual Value: -7.0
Constraint: constraint2
  Index: None, Dual Value: 5.0


In [11]:
# --- 1. Define Model Data ---
num_facilities = 2
num_customers = 3

facilities = range(num_facilities)
customers = range(num_customers)

# Cost to open facility j
opening_cost = {0: 100, 1: 120}

# Cost to ship from facility j to customer i
shipping_cost = {
    (0, 0): 10, (0, 1): 8, (0, 2): 12,
    (1, 0): 7, (1, 1): 11, (1, 2): 9
}

# Demand of customer i
demand = {0: 50, 1: 70, 2: 60}

# Capacity of facility j
capacity = {0: 100, 1: 150}

# --- 2. Create the Pyomo Model ---
model = pyo.ConcreteModel()

# --- 3. Define Variables ---
# Binary variable: 1 if facility j is opened, 0 otherwise
model.y = pyo.Var(facilities, within=pyo.Binary)

# Continuous variable: quantity shipped from facility j to customer i
model.x = pyo.Var(customers, facilities, bounds=(0, None))

# --- 4. Define Objective Function ---
# Minimize (opening costs + shipping costs)
model.obj = pyo.Objective(
    expr=sum(opening_cost[j] * model.y[j] for j in facilities) +
         sum(shipping_cost[i, j] * model.x[i, j]
             for i in customers for j in facilities),
    sense=pyo.minimize
)

# --- 5. Define Constraints ---

# Constraint 1: All demand must be met
model.demand_con = pyo.Constraint(customers, rule=
    lambda model, i: sum(model.x[i, j] for j in facilities) == demand[i]
)

# Constraint 2: Capacity at each opened facility
model.capacity_con = pyo.Constraint(facilities, rule=
    lambda model, j: sum(model.x[i, j] for i in customers) <= capacity[j] * model.y[j]
)

# Constraint 3: Cannot ship from a facility if it's not opened
# This is implicitly handled by capacity_con * y[j] where if y[j]=0, capacity becomes 0.

# IMPORTANT: Declare the 'dual' Suffix for duals (will be used in the second solve)
model.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

# --- 6. Solve the Full MIP with Gurobi ---
print("--- Solving the full MIP ---")
opt = SolverFactory('gurobi')
# You can add Gurobi specific options if needed, e.g., opt.options['TimeLimit'] = 60
results_mip = opt.solve(model, tee=True)

# Load results back into the model
model.load_results(results_mip)

# Check solver status
if (results_mip.solver.status == pyo.SolverStatus.ok and
    results_mip.solver.termination_condition == pyo.TerminationCondition.optimal):
    print("\n--- Optimal MIP Solution Found ---")
    print(f"Total Cost: {pyo.value(model.obj)}")

    print("\n--- Facility Decisions (y) ---")
    for j in facilities:
        print(f"Facility {j} opened: {pyo.value(model.y[j])}")

    print("\n--- Shipments (x) ---")
    for i in customers:
        for j in facilities:
            if pyo.value(model.x[i, j]) > 0.001: # Print significant shipments
                print(f"  Ship {pyo.value(model.x[i, j]):.2f} from F{j} to C{i}")

    # --- 7. Prepare for Dual Calculation (Fix Binary Variables and Re-solve as LP) ---
    print("\n--- Fixing binary variables and re-solving as LP for duals ---")

    # Deactivate the original objective to replace it temporarily
    model.obj.deactivate()

    # Create a new objective that is just the cost of the continuous part
    # Or, you can just keep the original objective, as fixing binary variables
    # effectively makes their contribution a constant.
    # The key is to make Pyomo treat 'y' as fixed parameters, not variables.

    # Fix the binary variables to their optimal MIP values
    for j in facilities:
        model.y[j].fix(pyo.value(model.y[j]))

    # Re-solve the model as an LP (Gurobi will detect it's an LP if only continuous vars are left)
    results_lp = opt.solve(model, tee=True)
    model.load_results(results_lp)

    if (results_lp.solver.status == pyo.SolverStatus.ok and
        results_lp.solver.termination_condition == pyo.TerminationCondition.optimal):
        print("\n--- Optimal LP Solution Found (with fixed binary variables) ---")
        print(f"Objective Value (fixed binary): {pyo.value(model.obj)}") # Should be same as original MIP obj

        print("\n--- Dual Values for Constraints ---")
        # Access dual values after the LP solve
        print("  Demand Constraints Duals:")
        for i in customers:
            dual_value = model.dual[model.demand_con[i]]
            print(f"    Customer {i} (Demand): {dual_value:.4f}")

        print("  Capacity Constraints Duals:")
        for j in facilities:
            dual_value = model.dual[model.capacity_con[j]]
            print(f"    Facility {j} (Capacity): {dual_value:.4f}")

        # You might also want to look at duals for variable bounds if they are binding
        # For simple bounds, this is usually straightforward.
        # e.g., model.dual[model.x[i,j]] for specific bounds, but it's less common.

    else:
        print("LP solve with fixed binaries did not find an optimal solution.")

else:
    print("\n--- Solver did not find an optimal MIP solution ---")
    print(f"Solver Status: {results_mip.solver.status}")
    print(f"Termination Condition: {results_mip.solver.termination_condition}")

KeyError: (2, 0)